In [1]:
import requests
from html.parser import HTMLParser
import re
from tqdm import tqdm
import json
import pandas as pd
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool
from selenium import webdriver
import html
from time import sleep

In [2]:
def get_ids(driver, url, t_sleep=10):
    driver.get(url)
    sleep(t_sleep)
    flag = 0
    find_count = 0
    need_count = driver.find_element_by_class_name('catalog').find_element_by_class_name('count-result').text
    num = int(need_count.split(' ')[1])
    result = []
    items = driver.find_elements_by_class_name("product-card")
    while(len(items) != num):
        items = driver.find_elements_by_class_name("product-card")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        if(len(items) == 60 and flag == 0):
            buttom = driver.find_element_by_xpath("/html/body/div[2]/div/div[7]/div/div[2]/div[4]/div[2]/div")
            sleep(10)
            buttom.click()
            flag = 1
    for item in items:
        classes = item.get_attribute('data-product').split()
        result.append(classes)
    return result



In [3]:
def books(ids):
    sleep(0.5)
    itog = []
    url_format = "https://www.chitai-gorod.ru/catalog/book/{card_id}/"
    for i in ids:
        url_param = {
            'card_id': i[0],
        }
        url_cur = url_format.format(**url_param)
        book_html = requests.get(url_cur).text
        soup = BeautifulSoup(book_html, 'html.parser')
        ra_in = soup.find('div', class_='rating').find('span').text
        rating, voice = ra_in.replace(')','').replace(' ', '').split('(')
        ff = [i.find_all('div') for i in soup.find('div', class_='product__props').find_all('div', class_='product-prop')]
        m = dict([list(map(lambda x: re.sub("\s", ' ', x.text.strip()), i)) for i in ff])
        cost = soup.find('div', class_= 'price')
        
        book_info = {
            'ID карточки': i[0],
            'Название': soup.find('h1', class_ = 'product__title').text,
            "Автор " : re.sub('\s', '',soup.find('a', class_='link product__author').text),
            "Рейтинг": rating,
            "Голоса": voice,
            "Цена": re.sub('\s', '',cost.text) if cost else None,
            "Обложка": soup.find('img', itemprop = 'image').get('src')
        }
        book_info.update(m)
        itog.append(book_info)
    return itog
        

In [4]:
def get_table():
    tables = []
    urls = ['https://www.chitai-gorod.ru/search/result.php?q=%EC%E0%EA%F1+%F4%F0%E0%E9',
            'https://www.chitai-gorod.ru/search/result.php?q=%DD%F0%E8%ED+%D5%E0%ED%F2%E5%F0',
             'https://www.chitai-gorod.ru/search/result.php?q=%C4%EC%E8%F2%F0%E8%E9+%C5%EC%E5%F6']
    for url in urls:
        with webdriver.Firefox() as driver:
            result = get_ids(driver, url)
        table = books(result)
        tables += table
    return tables

In [5]:
final_table = pd.DataFrame(get_table())

In [6]:
with open('hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    final_table.to_csv(f_csv, index=False)